In [1]:
#Connecting to google drive
from google.colab import drive 
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [2]:
%cd gdrive/MyDrive/Colab\ Notebooks/RSChallenge/RSLib
%ls

/content/gdrive/MyDrive/Colab Notebooks/RSChallenge/RSLib
RecSys_Course_AT_PoliMi-master/


In [3]:
!ls */*.py

RecSys_Course_AT_PoliMi-master/cython_example_FunkSVD.py
RecSys_Course_AT_PoliMi-master/cython_example_SLIM_MSE.py
RecSys_Course_AT_PoliMi-master/run_all_algorithms.py
RecSys_Course_AT_PoliMi-master/run_compile_all_cython.py
RecSys_Course_AT_PoliMi-master/run_hyperparameter_search.py
RecSys_Course_AT_PoliMi-master/run_test_recommenders.py


In [4]:
#Importing course's repositories
import sys
sys.path.append('/content/gdrive/MyDrive/Colab Notebooks/RSChallenge/RSLib/RecSys_Course_AT_PoliMi-master')

In [5]:
#To use skopt
! pip install --quiet lightgbm==2.2.3 scikit-optimize==0.9.0 neptune-client==0.4.132 neptune-contrib[monitoring]==0.25.0

     |████████████████████████████████| 100 kB 4.9 MB/s 
     |████████████████████████████████| 110 kB 43.0 MB/s 
     |████████████████████████████████| 67 kB 5.2 MB/s 
     |████████████████████████████████| 829 kB 31.7 MB/s 
     |████████████████████████████████| 53 kB 2.1 MB/s 
     |████████████████████████████████| 180 kB 56.5 MB/s 
     |████████████████████████████████| 106 kB 49.7 MB/s 
     |████████████████████████████████| 175 kB 58.0 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
     |████████████████████████████████| 99 kB 9.3 MB/s 
     |████████████████████████████████| 130 kB 48.1 MB/s 
     |████████████████████████████████| 67 kB 5.8 MB/s 
     |████████████████████████████████| 564 kB 33.4 MB/s 


In [86]:
#Importing libraries
import pandas as pd 
import scipy.sparse as sps
import numpy as np
import time
import matplotlib.pyplot as pyplot
import os
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Recommenders.Similarity.Compute_Similarity_Python import Compute_Similarity_Python
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender
from HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt
from HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
from Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender

print("DEBUG")

DEBUG


In [7]:
%cd ..
%ls

/content/gdrive/MyDrive/Colab Notebooks/RSChallenge
HYBRID.ipynb  Results/  RSData/  RSLib/  RSLib2/


In [8]:
#Loading dataframes
URM_dataframe = pd.read_csv('RSData/data_train.csv')
ICM_genre_dataframe=pd.read_csv('RSData/data_ICM_genre.csv')
ICM_subgenre_dataframe=pd.read_csv('RSData/data_ICM_subgenre.csv')
ICM_event_dataframe=pd.read_csv('RSData/data_ICM_event.csv')
ICM_channel_dataframe=pd.read_csv('RSData/data_ICM_channel.csv')

In [9]:
#Column Renaming
URM_dataframe.columns = ['User','Item','Interaction']
ICM_genre_dataframe.columns = ['Item','Feature','Val']
ICM_subgenre_dataframe.columns = ['Item','Feature','Val']
ICM_event_dataframe.columns = ['Item','Feature','Val']
ICM_channel_dataframe.columns = ['Item','Feature','Val']

In [10]:
#Converting datatype
URM_dataframe['Interaction'] = URM_dataframe['Interaction'].astype(int)
ICM_genre_dataframe['Val'] = ICM_genre_dataframe['Val'].astype(int)
ICM_subgenre_dataframe['Val'] = ICM_subgenre_dataframe['Val'].astype(int)
ICM_event_dataframe['Val'] = ICM_event_dataframe['Val'].astype(int)
ICM_channel_dataframe['Val'] = ICM_channel_dataframe['Val'].astype(int)

In [11]:
#Printing stats to see if everything went well
userID_unique = URM_dataframe["User"].unique()
itemID_unique = URM_dataframe["Item"].unique()
n_users = len(userID_unique)
n_items = len(itemID_unique)
n_interactions = len(URM_dataframe)

#Printing some stats
print("Number of users: " + str(n_users))
print("Number of items: " + str(n_items))
print("Number of interactions: " + str(n_interactions))

Number of users: 13650
Number of items: 18059
Number of interactions: 5285664


In [12]:
#Checking if re-assignment of ID's is needed
print('The maxID for user {}, The maxID for item {}'.format(max(URM_dataframe.User),max(URM_dataframe.Item)))

The maxID for user 13649, The maxID for item 18058


Re-assignment not needed

In [13]:
#Converting the URM into a sparse matrix
URM_sparse = sps.coo_matrix((URM_dataframe['Interaction'].values,(URM_dataframe['User'].values,URM_dataframe['Item'].values)))
URM_csr = URM_sparse.tocsr()
URM_csr

<13650x18059 sparse matrix of type '<class 'numpy.longlong'>'
	with 5285664 stored elements in Compressed Sparse Row format>

In [14]:
#Splitting data into training and validation set (Test set is the one on Kaggle)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_csr, train_percentage = 0.80)
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

EvaluatorHoldout: Ignoring 13646 ( 0.0%) Users that have less than 1 test interactions


This requires a lot of time, let's first try with some random values ...

In [15]:
recommender = SLIMElasticNetRecommender(URM_train)

In [16]:
recommender.fit(l1_ratio = 0.0002204448537910578, alpha = 0.079724114437381, topK = 3100) # 47.33%

SLIMElasticNetRecommender: Processed 1478 ( 8.2%) in 5.00 min. Items per second: 4.92
SLIMElasticNetRecommender: Processed 3027 (16.8%) in 10.01 min. Items per second: 5.04
SLIMElasticNetRecommender: Processed 4583 (25.4%) in 15.01 min. Items per second: 5.09
SLIMElasticNetRecommender: Processed 6151 (34.1%) in 20.01 min. Items per second: 5.12
SLIMElasticNetRecommender: Processed 7733 (42.8%) in 25.01 min. Items per second: 5.15
SLIMElasticNetRecommender: Processed 9307 (51.5%) in 30.02 min. Items per second: 5.17
SLIMElasticNetRecommender: Processed 10902 (60.4%) in 35.02 min. Items per second: 5.19
SLIMElasticNetRecommender: Processed 12485 (69.1%) in 40.02 min. Items per second: 5.20
SLIMElasticNetRecommender: Processed 14070 (77.9%) in 45.02 min. Items per second: 5.21
SLIMElasticNetRecommender: Processed 15656 (86.7%) in 50.02 min. Items per second: 5.22
SLIMElasticNetRecommender: Processed 17226 (95.4%) in 55.02 min. Items per second: 5.22
SLIMElasticNetRecommender: Processed 18

In [17]:
recommender2 = IALSRecommender(URM_train)

In [90]:
recommender2.fit(epochs = 32, num_factors = 50, alpha = 1.011955985422186, reg = 0.0006510527003718695, init_mean = 0.4902658701038708, init_std = 0.0542837791767091) # Così ottengo: 0.418 MAP: 0.2335376

IALSRecommender: Epoch 1 of 32. Elapsed time 11.76 sec
IALSRecommender: Epoch 2 of 32. Elapsed time 23.29 sec
IALSRecommender: Epoch 3 of 32. Elapsed time 34.85 sec
IALSRecommender: Epoch 4 of 32. Elapsed time 48.90 sec
IALSRecommender: Epoch 5 of 32. Elapsed time 1.04 min
IALSRecommender: Epoch 6 of 32. Elapsed time 1.23 min
IALSRecommender: Epoch 7 of 32. Elapsed time 1.52 min
IALSRecommender: Epoch 8 of 32. Elapsed time 1.99 min
IALSRecommender: Epoch 9 of 32. Elapsed time 2.46 min
IALSRecommender: Epoch 10 of 32. Elapsed time 2.81 min
IALSRecommender: Epoch 11 of 32. Elapsed time 3.00 min
IALSRecommender: Epoch 12 of 32. Elapsed time 3.19 min
IALSRecommender: Epoch 13 of 32. Elapsed time 3.39 min
IALSRecommender: Epoch 14 of 32. Elapsed time 3.58 min
IALSRecommender: Epoch 15 of 32. Elapsed time 3.78 min
IALSRecommender: Epoch 16 of 32. Elapsed time 3.97 min
IALSRecommender: Epoch 17 of 32. Elapsed time 4.16 min
IALSRecommender: Epoch 18 of 32. Elapsed time 4.35 min
IALSRecommender

In [87]:
recommenderSVD = PureSVDRecommender(URM_train)

In [89]:
recommenderSVD.fit(num_factors = 28)

PureSVDRecommender: Computing SVD decomposition...
PureSVDRecommender: Computing SVD decomposition... done in 3.84 sec


In [98]:
from Recommenders.BaseRecommender import BaseRecommender

class DifferentLossScoresHybridRecommender(BaseRecommender):
    """ ScoresHybridRecommender
    Hybrid of two prediction scores R = R1/norm*alpha + R2/norm*(1-alpha) where R1 and R2 come from
    algorithms trained on different loss functions.

    """

    RECOMMENDER_NAME = "DifferentLossScoresHybridRecommender"


    def __init__(self, URM_train, recommender_1, recommender_2, recommender_3):
        super(DifferentLossScoresHybridRecommender, self).__init__(URM_train)

        self.URM_train = sps.csr_matrix(URM_train)
        self.recommender_1 = recommender_1
        self.recommender_2 = recommender_2
        self.recommender_3 = recommender_3
        
        
        
    def fit(self, norm, alpha = 0.5, beta = 0.5, gamma = 0.5):

        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma
        self.norm = norm


    def _compute_item_score(self, user_id_array, items_to_compute):
        
        item_weights_1 = self.recommender_1._compute_item_score(user_id_array)
        item_weights_2 = self.recommender_2._compute_item_score(user_id_array)
        item_weights_3 = self.recommender_3._compute_item_score(user_id_array)

        norm_item_weights_1 = LA.norm(item_weights_1, self.norm)
        norm_item_weights_2 = LA.norm(item_weights_2, self.norm)
        norm_item_weights_3 = LA.norm(item_weights_3, self.norm)
        
        
        if norm_item_weights_1 == 0:
            raise ValueError("Norm {} of item weights for recommender 1 is zero. Avoiding division by zero".format(self.norm))
        
        if norm_item_weights_2 == 0:
            raise ValueError("Norm {} of item weights for recommender 2 is zero. Avoiding division by zero".format(self.norm))
        
        item_weights = item_weights_1 / norm_item_weights_1 * self.alpha + item_weights_2 / norm_item_weights_2 * (self.beta) + item_weights_3 / norm_item_weights_3 * (self.gamma)

        return item_weights

#HYBRID SLIM + PURE SVD + MF IALS

In [138]:
hybrid = DifferentLossScoresHybridRecommender(URM_train, recommender, recommender2, recommenderSVD)
hybrid.fit(norm = -np.inf, alpha = 0.7, beta = 0.28, gamma = 0.02)
result_df, _ = evaluator_validation.evaluateRecommender(hybrid)
display(result_df)

EvaluatorHoldout: Processed 13646 (100.0%) in 34.00 sec. Users per second: 401


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,NOVELTY,AVERAGE_POPULARITY,DIVERSITY_MEAN_INTER_LIST,DIVERSITY_HERFINDAHL,COVERAGE_ITEM,COVERAGE_ITEM_CORRECT,COVERAGE_USER,COVERAGE_USER_CORRECT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,,,,,
10,0.403356,0.405193,0.0720257,0.250241,0.251137,0.648708,0.416973,0.122226,0.972666,1.2515,0.00552037,0.545722,0.918981,0.991891,0.0667811,0.0492829,0.999707,0.972381,0.0108116,7.89693,0.992274,0.0436075,0.637364,2.66379,0.0259737


Migliore fin'ora: 0.7 0.28 0.02

#SLIM TOK K 3000 NORM 2

In [56]:
# 0.7
hybrid = DifferentLossScoresHybridRecommender(URM_train, recommender, recommender2)
hybrid.fit(norm = -np.inf, alpha = 0.7)
result_df, _ = evaluator_validation.evaluateRecommender(hybrid)
result_df

EvaluatorHoldout: Processed 13646 (100.0%) in 32.40 sec. Users per second: 421


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,NOVELTY,AVERAGE_POPULARITY,DIVERSITY_MEAN_INTER_LIST,DIVERSITY_HERFINDAHL,COVERAGE_ITEM,COVERAGE_ITEM_CORRECT,COVERAGE_USER,COVERAGE_USER_CORRECT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,,,,,
10,0.403298,0.405137,0.0720926,0.250358,0.251239,0.649596,0.417129,0.12232,0.9723,1.25272,0.00552297,0.544703,0.919408,0.991934,0.0679993,0.0502243,0.999707,0.972015,0.0109371,7.91046,0.992317,0.0441139,0.638456,2.65881,0.025986


In [60]:
# 0.75
hybrid = DifferentLossScoresHybridRecommender(URM_train, recommender, recommender2)
hybrid.fit(norm = -np.inf, alpha = 0.75)
result_df, _ = evaluator_validation.evaluateRecommender(hybrid)
result_df

EvaluatorHoldout: Processed 13646 (100.0%) in 36.35 sec. Users per second: 375


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,NOVELTY,AVERAGE_POPULARITY,DIVERSITY_MEAN_INTER_LIST,DIVERSITY_HERFINDAHL,COVERAGE_ITEM,COVERAGE_ITEM_CORRECT,COVERAGE_USER,COVERAGE_USER_CORRECT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,,,,,
10,0.403444,0.405272,0.0720107,0.250325,0.25119,0.649035,0.417026,0.122208,0.972006,1.25172,0.00552046,0.54632,0.918299,0.991823,0.0686085,0.050335,0.999707,0.971722,0.0108458,7.89497,0.992206,0.0437454,0.637206,2.66671,0.0259742


In [84]:
# 0.79
hybrid = DifferentLossScoresHybridRecommender(URM_train, recommender, recommender2)
hybrid.fit(norm = -np.inf, alpha = 0.79)
result_df, _ = evaluator_validation.evaluateRecommender(hybrid)
result_df

EvaluatorHoldout: Processed 13646 (100.0%) in 35.09 sec. Users per second: 389


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,NOVELTY,AVERAGE_POPULARITY,DIVERSITY_MEAN_INTER_LIST,DIVERSITY_HERFINDAHL,COVERAGE_ITEM,COVERAGE_ITEM_CORRECT,COVERAGE_USER,COVERAGE_USER_CORRECT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,,,,,
10,0.403496,0.405314,0.0720479,0.250354,0.251205,0.64874,0.417013,0.122264,0.97208,1.25155,0.00551869,0.547518,0.91742,0.991735,0.0689407,0.0505011,0.999707,0.971795,0.0107727,7.8828,0.992118,0.0434506,0.636224,2.67255,0.0259659


In [57]:
# 0.8
hybrid = DifferentLossScoresHybridRecommender(URM_train, recommender, recommender2)
hybrid.fit(norm = -np.inf, alpha = 0.8)
result_df, _ = evaluator_validation.evaluateRecommender(hybrid)
result_df

EvaluatorHoldout: Processed 13646 (100.0%) in 32.34 sec. Users per second: 422


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,NOVELTY,AVERAGE_POPULARITY,DIVERSITY_MEAN_INTER_LIST,DIVERSITY_HERFINDAHL,COVERAGE_ITEM,COVERAGE_ITEM_CORRECT,COVERAGE_USER,COVERAGE_USER_CORRECT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,,,,,
10,0.403474,0.405291,0.0720331,0.250378,0.251231,0.649071,0.417058,0.122242,0.97208,1.25188,0.0055184,0.547716,0.917274,0.991721,0.0692729,0.0505565,0.999707,0.971795,0.01076,7.88064,0.992104,0.0433994,0.636049,2.67352,0.0259645


In [85]:
# 0.81
hybrid = DifferentLossScoresHybridRecommender(URM_train, recommender, recommender2)
hybrid.fit(norm = -np.inf, alpha = 0.81)
result_df, _ = evaluator_validation.evaluateRecommender(hybrid)
result_df

EvaluatorHoldout: Processed 13646 (100.0%) in 32.77 sec. Users per second: 416


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,NOVELTY,AVERAGE_POPULARITY,DIVERSITY_MEAN_INTER_LIST,DIVERSITY_HERFINDAHL,COVERAGE_ITEM,COVERAGE_ITEM_CORRECT,COVERAGE_USER,COVERAGE_USER_CORRECT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,,,,,
10,0.403334,0.405156,0.0720147,0.250325,0.251178,0.649196,0.416984,0.122209,0.972153,1.25183,0.00551819,0.547912,0.917126,0.991706,0.0694391,0.0506119,0.999707,0.971868,0.0107458,7.8784,0.992089,0.0433421,0.635869,2.67448,0.0259635


In [61]:
# 0.85
hybrid = DifferentLossScoresHybridRecommender(URM_train, recommender, recommender2)
hybrid.fit(norm = -np.inf, alpha = 0.85)
result_df, _ = evaluator_validation.evaluateRecommender(hybrid)
result_df

EvaluatorHoldout: Processed 13646 (100.0%) in 33.60 sec. Users per second: 406


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,NOVELTY,AVERAGE_POPULARITY,DIVERSITY_MEAN_INTER_LIST,DIVERSITY_HERFINDAHL,COVERAGE_ITEM,COVERAGE_ITEM_CORRECT,COVERAGE_USER,COVERAGE_USER_CORRECT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,,,,,
10,0.402975,0.40478,0.071949,0.250116,0.250967,0.649743,0.416762,0.122098,0.972006,1.25163,0.00551661,0.54896,0.916395,0.991633,0.0700482,0.0501689,0.999707,0.971722,0.0106816,7.86778,0.992016,0.0430832,0.635012,2.67959,0.025956


#Building the final recommender

In [62]:
recSLIM = SLIMElasticNetRecommender(URM_sparse)

In [63]:
recMFIALS = IALSRecommender(URM_sparse)

In [64]:
recSLIM.fit(l1_ratio = 0.0002204448537910578, alpha = 0.079724114437381, topK = 3100)

SLIMElasticNetRecommender: Processed 1124 ( 6.2%) in 5.00 min. Items per second: 3.74
SLIMElasticNetRecommender: Processed 2321 (12.9%) in 10.00 min. Items per second: 3.87
SLIMElasticNetRecommender: Processed 3521 (19.5%) in 15.01 min. Items per second: 3.91
SLIMElasticNetRecommender: Processed 4711 (26.1%) in 20.01 min. Items per second: 3.92
SLIMElasticNetRecommender: Processed 5892 (32.6%) in 25.01 min. Items per second: 3.93
SLIMElasticNetRecommender: Processed 7081 (39.2%) in 30.01 min. Items per second: 3.93
SLIMElasticNetRecommender: Processed 8269 (45.8%) in 35.01 min. Items per second: 3.94
SLIMElasticNetRecommender: Processed 9467 (52.4%) in 40.01 min. Items per second: 3.94
SLIMElasticNetRecommender: Processed 10674 (59.1%) in 45.02 min. Items per second: 3.95
SLIMElasticNetRecommender: Processed 11868 (65.7%) in 50.02 min. Items per second: 3.95
SLIMElasticNetRecommender: Processed 13065 (72.3%) in 55.02 min. Items per second: 3.96
SLIMElasticNetRecommender: Processed 1425

In [65]:
recMFIALS.fit(epochs = 32, num_factors = 50, alpha = 1.011955985422186, reg = 0.0006510527003718695, init_mean = 0.4902658701038708, init_std = 0.0542837791767091)

IALSRecommender: Epoch 1 of 32. Elapsed time 13.08 sec
IALSRecommender: Epoch 2 of 32. Elapsed time 25.98 sec
IALSRecommender: Epoch 3 of 32. Elapsed time 38.87 sec
IALSRecommender: Epoch 4 of 32. Elapsed time 51.74 sec
IALSRecommender: Epoch 5 of 32. Elapsed time 1.08 min
IALSRecommender: Epoch 6 of 32. Elapsed time 1.29 min
IALSRecommender: Epoch 7 of 32. Elapsed time 1.51 min
IALSRecommender: Epoch 8 of 32. Elapsed time 1.72 min
IALSRecommender: Epoch 9 of 32. Elapsed time 1.94 min
IALSRecommender: Epoch 10 of 32. Elapsed time 2.15 min
IALSRecommender: Epoch 11 of 32. Elapsed time 2.37 min
IALSRecommender: Epoch 12 of 32. Elapsed time 2.59 min
IALSRecommender: Epoch 13 of 32. Elapsed time 2.80 min
IALSRecommender: Epoch 14 of 32. Elapsed time 3.02 min
IALSRecommender: Epoch 15 of 32. Elapsed time 3.24 min
IALSRecommender: Epoch 16 of 32. Elapsed time 3.45 min
IALSRecommender: Epoch 17 of 32. Elapsed time 3.67 min
IALSRecommender: Epoch 18 of 32. Elapsed time 3.89 min
IALSRecommender

In [139]:
recommenderSVD = PureSVDRecommender(URM_sparse)

In [140]:
recommenderSVD.fit(num_factors=28)

PureSVDRecommender: Computing SVD decomposition...
PureSVDRecommender: Computing SVD decomposition... done in 4.73 sec


In [76]:
hybrid_SLIM_MFIALS = DifferentLossScoresHybridRecommender(URM_sparse, recSLIM, recMFIALS)
hybrid_SLIM_MFIALS.fit(norm = -np.inf, alpha = 0.8)

In [142]:
hybrid_SLIM_MFIALS_SVD = DifferentLossScoresHybridRecommender(URM_sparse, recSLIM, recMFIALS, recommenderSVD)
hybrid_SLIM_MFIALS_SVD.fit(norm = -np.inf, alpha = 0.7, beta=0.28, gamma=0.02)

In [143]:
%cd RSChallenge/
%ls

/content/gdrive/My Drive/Colab Notebooks/RSChallenge
HYBRID1.ipynb  Results/  RSLib/   submission_HYBRID_1.csv
HYBRID2.ipynb  RSData/   RSLib2/  submission_HYBRID_2.csv


In [144]:
target_users = pd.read_csv('RSData/data_target_users_test.csv')

In [145]:
final_dict = {}
start_time = time.time()

for user in target_users['user_id']:
  final_dict[user] = hybrid_SLIM_MFIALS_SVD.recommend(user,cutoff=10)
end_time = time.time()
print('Time needed is {}seconds'.format(end_time-start_time))

Time needed is 83.88834977149963seconds


In [146]:
item_list = []
for user in target_users['user_id']:
  item_list.append(' '.join(str(e) for e in final_dict[user]))

In [147]:
submission_df = pd.DataFrame({'user_id' : target_users['user_id'],'item_list' : item_list })
submission_df['item_list'] = submission_df['item_list'].astype(str).str.strip('[]')
submission_df

,user_id,item_list
0,0,3899 5433 16365 16359 8465 6061 2243 9130 640 ...
1,1,3473 940 14400 2118 16012 9275 16365 640 10118...
2,2,6177 1551 17330 17022 14229 16365 15119 4607 2...
3,3,8194 15551 15082 16359 960 8371 15507 5885 598...
4,4,8194 10586 9871 2152 6177 5299 10954 11836 601...
...,...,...
13645,13645,6177 11491 3200 1259 8898 11576 4078 11303 776...
13646,13646,3473 7867 8635 16640 17266 17623 409 11504 176...
13647,13647,1846 7716 7606 11227 8635 14868 1186 16846 114...
13648,13648,6177 8635 13295 6214 14181 7295 17936 5273 143...


In [148]:
%ls
%cd ..

HYBRID1.ipynb  Results/  RSLib/   submission_HYBRID_1.csv
HYBRID2.ipynb  RSData/   RSLib2/  submission_HYBRID_2.csv
/content/gdrive/MyDrive/Colab Notebooks


In [149]:
submission_df.to_csv('RSChallenge/submission_HYBRID_3.csv', index=False)

Controlliamo quanto le recomendations dell'ibrido a 3 sono diverse dalle recommendations dell'ibrido a 2.

In [150]:
%ls

 AN2DL/                                      Model_aug_no_transf.ipynb
'CONV ESAMI.ipynb'                           RS/
'Copia di DataSeries-Part2-Modeling.ipynb'   RSChallenge/
 DM/                                         SDA/
 gra.ipynb


In [151]:
HYB2 = pd.read_csv('RSChallenge/submission_HYBRID_2.csv') # Loading DF
HYB3 = pd.read_csv('RSChallenge/submission_HYBRID_3.csv') # Loading DF

In [152]:
HYB2

,user_id,item_list
0,0,3899 5433 16365 8465 16359 2243 6061 9130 640 ...
1,1,3473 940 14400 16012 2118 640 9275 16365 10118...
2,2,6177 1551 17022 17330 14229 16365 2846 15119 4...
3,3,8194 15551 15082 16359 8371 960 15507 16444 58...
4,4,8194 10586 6177 2152 9871 10954 5299 6011 1649...
...,...,...
13645,13645,6177 11491 3200 8898 1259 11576 7762 11303 407...
13646,13646,3473 7867 8635 16640 17266 17623 409 11504 176...
13647,13647,1846 7716 7606 11227 8635 14868 1186 11491 168...
13648,13648,6177 8635 13295 6214 7295 14358 14181 17936 52...


In [153]:
HYB3

,user_id,item_list
0,0,3899 5433 16365 16359 8465 6061 2243 9130 640 ...
1,1,3473 940 14400 2118 16012 9275 16365 640 10118...
2,2,6177 1551 17330 17022 14229 16365 15119 4607 2...
3,3,8194 15551 15082 16359 960 8371 15507 5885 598...
4,4,8194 10586 9871 2152 6177 5299 10954 11836 601...
...,...,...
13645,13645,6177 11491 3200 1259 8898 11576 4078 11303 776...
13646,13646,3473 7867 8635 16640 17266 17623 409 11504 176...
13647,13647,1846 7716 7606 11227 8635 14868 1186 16846 114...
13648,13648,6177 8635 13295 6214 14181 7295 17936 5273 143...


In [154]:
HYB2_ll = HYB2.values.tolist() 
HYB2_list = HYB2_ll[0][1].split()
HYB3_ll = HYB3.values.tolist() 
HYB3_list = HYB3_ll[0][1].split()

In [155]:
num10 = 0
num9 = 0
num8 = 0
num7 = 0
num6 = 0
num5 = 0
num4 = 0
num3 = 0
num2 = 0
num1 = 0
num0 = 0

for user in target_users['user_id']:
  count = 0
  HYB2_list = HYB2_ll[user][1].split()
  HYB3_list = HYB3_ll[user][1].split()
  for k in range(10):
    if(HYB2_list[k] in HYB3_list):
      count = count + 1
  
  if count == 10: num10=num10+1
  if count == 9: num9=num9+1
  if count == 8: num8=num8+1
  if count == 7: num7=num7+1
  if count == 6: num6=num6+1
  if count == 5: num5=num5+1
  if count == 4: num4=num4+1
  if count == 3: num3=num3+1
  if count == 2: num2=num2+1
  if count == 1: num1=num1+1

print(num10)
print(num9)
print(num8)
print(num7)
print(num6)
print(num5)
print(num4)
print(num3)
print(num2)
print(num1)

9350
4100
196
4
0
0
0
0
0
0
